# Kinetic Mechanism Pathway Analyzer

This analyzer is designed to provide a better user experience and improved analyzing capacity (e.g., large mechanisms) compared with Chemkin mechanism analyzer. This analyzer takes reaction ROP file (e.g. Chemkin ckcsv file), builds a reaction network, helps answer

- if the mechanism has certain pathway (from start species to end species)

- what is the biggest flux from/towards a certain species

When using this script, please pay attention to the sections marked with **[user input]** which require user to specify certain input parameters.

In [1]:
from rmgpy.rmg.model import CoreEdgeReactionModel
from rmgpy.chemkin import loadChemkinFile
import os

## [user input] Prepare files

Please add 

- annotated chemkin files (mechanism and species dictionary) and 

- chemkin simulation ckcsv file 

to a folder with name stored in `mech` variable in the next cell.

In [2]:
mech = 'minimal'

In [3]:
mechPath = os.path.join('./', 'data', 'pathway_analysis', mech)
chemkinPath= os.path.join(mechPath, 'chem.inp')
dictionaryPath = os.path.join(mechPath, 'species_dictionary.txt')
ckcsvPath= mechPath + '/CKSoln.ckcsv'

model = CoreEdgeReactionModel()
model.core.species, model.core.reactions = loadChemkinFile(chemkinPath,dictionaryPath)

In [4]:
# generate paris for reactions that don't have flux pairs
for rxn in model.core.reactions:
    if not rxn.pairs: rxn.generatePairs()

## Create reaction network

In [5]:
import networkx as nx
import matplotlib.pyplot as plt
from rmgpy.tools.extractInfoFromckcsv import getConcentrationDictFromCKCSV, getROPFromCKCSV, getFluxGraphEdgesDict
from rmgpy.chemkin import getSpeciesIdentifier
from IPython.display import display
import numpy as np
from rmgpy.rmg.pdep import PDepReaction
%matplotlib inline

### step1: prepare a dict for graph edges

In [6]:
firstColDict, spc_mf_dict = getConcentrationDictFromCKCSV(ckcsvPath)
first_col_dict, spc_total_rop_dict, spc_rop_dict = getROPFromCKCSV(ckcsvPath)

graph_edges_dict = getFluxGraphEdgesDict(spc_rop_dict, model.core.reactions)

graph_edges_dict_simple = {}
for pair in graph_edges_dict:
    node1 = getSpeciesIdentifier(pair[0])
    node2 = getSpeciesIdentifier(pair[1])
    graph_edges_dict_simple[(node1, node2)] = graph_edges_dict[pair]

### [user input] step2: pick a time to analyze pathways

Please specify 

- the time point you want to investigate

- the chemkin name of your initial species/reactant

In [7]:
time_investigated = 0.000001 # hour
reactant_name = 'ethane(1)'
timepoint_index = (np.abs(firstColDict['Time_(sec)']-time_investigated*3600)).argmin()
print "Investiageted time point is {0} secs".format(firstColDict['Time_(sec)'][timepoint_index])
print "At this moment, the reactant's mole fraction is {}.".format(spc_mf_dict[reactant_name][timepoint_index])

# create DiGraph for this moment
G = nx.DiGraph()
for pair in graph_edges_dict:
    node1 = getSpeciesIdentifier(pair[0])
    node2 = getSpeciesIdentifier(pair[1])
    e_rawdata = graph_edges_dict[pair]
    total_flux = 0
    for rxn in e_rawdata:
        total_flux += e_rawdata[rxn][timepoint_index]
    if total_flux >= 0:
        G.add_edge(node2, node1, {"total_flux":total_flux}) # in G, positive means production of node1
    else: 
        G.add_edge(node1, node2, {"total_flux":-total_flux}) # in G, negative means consumption of node1      

Investiageted time point is 0.003731082 secs
At this moment, the reactant's mole fraction is 0.2634996.


## [user input] Functionality 1: find paths between two species

Pleae specify

- source species

- target species

In [8]:
src_species = 'ethane(1)'
tgt_species = 'C#C(25)'

## output1: sort out all the paths and most significant one

In [9]:
paths = list(nx.all_simple_paths(G, source=src_species, target=tgt_species, cutoff=5))

In [10]:
path_fluxes = []
print '================================================'
print 'Path index \t Flux \t\t Path steps'
print ''

for i, path in enumerate(paths):
    
    path_steps = len(path) - 1
    fluxes = [G[path[step]][path[step+1]]['total_flux'] for step in range(path_steps) ]
    path_fluxes.append(min(fluxes))
    print 'Path #{0}: \t {1:.6f} \t {2}'.format(i, min(fluxes), path)
sorted_path_fluxes = sorted(path_fluxes)

print ''
print '============================================'
print '\t Most Significant Path '
print '============================================'
print 'Path index \t Flux'
print 'Path #{0}: \t {1:.6f} '.format(path_fluxes.index(sorted_path_fluxes[-1]), sorted_path_fluxes[-1])

Path index 	 Flux 		 Path steps

Path #0: 	 0.000003 	 ['ethane(1)', 'C2H5(3)', 'H(4)', 'C2H4(8)', 'C2H3(13)', 'C#C(25)']
Path #1: 	 0.000168 	 ['ethane(1)', 'C2H5(3)', 'C2H4(8)', 'C2H3(13)', 'C#C(25)']

	 Most Significant Path 
Path index 	 Flux
Path #1: 	 0.000168 


### [user input] output2: visualize the path of interest

Please specify

- the path index you want to visualize

In [18]:
path_index_investigate = 0
path = paths[path_index_investigate]
path_steps = len(path) - 1
print ''
print '\t Pathway Report '
print '======================================'
print 'The pathway you are intested in has {0} steps.'.format(len(path))
for step in range(path_steps):
    step_pair = (path[step], path[step+1])
    h_abs_rxns = []
    disp_rxns = []
    Pdep_rxns = []
    
    print ""
    print "Step{0} \t\t\t\t Flux".format(step+1)
    print '******************************************'
    
    print "{1} --> {2} \t\t {3}".\
    format(step+1, step_pair[0], step_pair[1], G[step_pair[0]][step_pair[1]]['total_flux'])
    
    print '******************************************'
    if step_pair not in graph_edges_dict_simple:
        step_pair = (step_pair[1], step_pair[0])
    
    print ''
    for rxn in graph_edges_dict_simple[step_pair]:
        if isinstance(rxn, PDepReaction):
            Pdep_rxns.append(rxn)
        elif rxn.family == "H_Abstraction":
            h_abs_rxns.append(rxn)
        elif rxn.family == "Disproportionation":
            disp_rxns.append(rxn)
        else:
            print "Example reaction: rxn#{0}".format(rxn.index)
            print ''
            print str(rxn)
            display(rxn)
    if len(h_abs_rxns) > 0: 
        
        print "Example H_Abstraction: rxn#{0} (1/{1} H_Abs): ".format(h_abs_rxns[0].index, len(h_abs_rxns)) 
        print ''
        print str(h_abs_rxns[0])
        display(h_abs_rxns[0])
    if len(disp_rxns) > 0: 
        
        print "Example Disproportionation: rxn#{0} (1/{1} Disp): ".format(disp_rxns[0].index, len(disp_rxns)) 
        print ''
        print str(disp_rxns[0])
        display(disp_rxns[0])
    if len(Pdep_rxns) > 0: 
        
        print "Example Pressure-dependent Rxn: rxn#{0} (1/{1} Pdep_rxns): ".format(Pdep_rxns[0].index, len(Pdep_rxns)) 
        print ''
        print str(Pdep_rxns[0])
        display(Pdep_rxns[0])


	 Pathway Report 
The pathway you are intested in has 6 steps.

Step1 				 Flux
******************************************
ethane(1) --> C2H5(3) 		 0.0005349806659
******************************************

Example reaction: rxn#3

C[CH2](3) + [H](4) <=> ethane(1)


Example H_Abstraction: rxn#2 (1/3 H_Abs): 

[CH3](2) + ethane(1) <=> C[CH2](3) + C(6)


Example Disproportionation: rxn#7 (1/2 Disp): 

C[CH2](3) + C[CH2](3) <=> C=C(8) + ethane(1)



Step2 				 Flux
******************************************
C2H5(3) --> H(4) 		 0.0005009196
******************************************

Example reaction: rxn#5

C=C(8) + [H](4) <=> C[CH2](3)



Step3 				 Flux
******************************************
H(4) --> C2H4(8) 		 3.402726e-06
******************************************

Example reaction: rxn#13

[CH]=C(13) + [H](4) <=> C=C(8)



Step4 				 Flux
******************************************
C2H4(8) --> C2H3(13) 		 0.0001682719323
******************************************

Example reaction: rxn#13

[CH]=C(13) + [H](4) <=> C=C(8)


Example H_Abstraction: rxn#14 (1/3 H_Abs): 

[CH]=C(13) + C(6) <=> C=C(8) + [CH3](2)


Example Disproportionation: rxn#17 (1/2 Disp): 

[CH]=C(13) + C[CH2](3) <=> C=C(8) + C=C(8)



Step5 				 Flux
******************************************
C2H3(13) --> C#C(25) 		 0.0001685005362
******************************************

Example reaction: rxn#18

C#C(25) + [H](4) <=> [CH]=C(13)


Example Disproportionation: rxn#22 (1/4 Disp): 

[CH]=C(13) + [CH]=C(13) <=> C=C(8) + C#C(25)


## [user input] Functionality 2: find paths from a particular species

Please specify

- source species

- depth of pathway you want search, e.g., depth = 2 means search pathways 2 steps down from source species

- path_top_list, e.g., [0, 3] means 1st step of the pathway takes the most significant branch and 2nd step would be the third most siginificant branch

In [19]:
source = "ethane(1)"
depth = 2
path_top_list = [0, 0]

In [23]:
current_node = source

print ''
print '\t Pathway Report '
print '======================================'
print 'The pathway you are intested in has {0} steps.'.format(depth)
for step in range(depth):
    print "\n"    
    nextNode_flux_list = [(next_node, G[current_node][next_node]['total_flux']) for next_node in G[current_node]]
    sorted_nextNode_flux_list = sorted(nextNode_flux_list, key=lambda tup: -tup[1])
    
    # choose the top one as next node
    tup = sorted_nextNode_flux_list[path_top_list[step]]
    next_node = tup[0]
    step_flux = tup[1]
    
    print ""
    print "Step{0} \t\t\t\t Flux".format(step+1)
    print '******************************************'
    
    print "{0} --> {1} \t\t {2}".\
    format(current_node, next_node, step_flux)
    print '******************************************'
    
    step_pair = (current_node, next_node)
    if step_pair not in graph_edges_dict_simple:
        step_pair = (next_node, current_node)
    
    h_abs_rxns = []
    disp_rxns = []
    Pdep_rxns = []
    for rxn in graph_edges_dict_simple[step_pair]:
        if isinstance(rxn, PDepReaction):
            Pdep_rxns.append(rxn)
        elif rxn.family == "H_Abstraction":
            h_abs_rxns.append(rxn)
        elif rxn.family == "Disproportionation":
            disp_rxns.append(rxn)
        else:
            print ''
            print "Example reaction: rxn#{0}".format(rxn.index)
            print ''
            print "{0}: flux = {1} ".format(str(rxn), graph_edges_dict_simple[step_pair][rxn][timepoint_index])
            display(rxn)
    if len(h_abs_rxns) > 0:
        print ''
        print "Example H_Abstraction: rxn#{0}(1/{1} H_Abs)".format(h_abs_rxns[0].index, len(h_abs_rxns))
        print ''
        print str(h_abs_rxns[0])
        display(h_abs_rxns[0])
    if len(disp_rxns) > 0: 
        print ''
        print "Example Disproportionation: rxn#{0}(1/{1} Disp)".format(disp_rxns[0].index, len(disp_rxns))
        print ''
        print str(disp_rxns[0])
        display(disp_rxns[0])
    if len(Pdep_rxns) > 0: 
        print ''
        print "Example Pressure-dependent Rxn: rxn#{0} (1/{1} Pdep_rxns): ".format(Pdep_rxns[0].index, len(Pdep_rxns)) 
        print ''
        print str(Pdep_rxns[0])
        display(Pdep_rxns[0])
    
    current_node = next_node


	 Pathway Report 
The pathway you are intested in has 2 steps.



Step1 				 Flux
******************************************
ethane(1) --> C2H5(3) 		 0.0005349806659
******************************************

Example reaction: rxn#3

C[CH2](3) + [H](4) <=> ethane(1): flux = 2.844234e-05 



Example H_Abstraction: rxn#2(1/3 H_Abs)

[CH3](2) + ethane(1) <=> C[CH2](3) + C(6)



Example Disproportionation: rxn#7(1/2 Disp)

C[CH2](3) + C[CH2](3) <=> C=C(8) + ethane(1)





Step2 				 Flux
******************************************
C2H5(3) --> C2H4(8) 		 0.000536020903
******************************************

Example reaction: rxn#5

C=C(8) + [H](4) <=> C[CH2](3): flux = 0.0005009196 



Example Disproportionation: rxn#17(1/4 Disp)

[CH]=C(13) + C[CH2](3) <=> C=C(8) + C=C(8)


### More detail for the H_Abstration and Disproportionation in last step of the pathway

In [24]:
print ''
print 'The step you see currently is between the two species {0}'.format(step_pair)
print ''
print 'Positive flux in below means positive production of 1st node of the pair.'
print ''
print 'This step have {0} H_Abstration and Disproportionation in total.'.format(len(h_abs_rxns + disp_rxns))

flux_threshold = 1e-9

print ''
print 'TO avoid too much printout, the reactions shown below have fluxes above {0}.'.format(flux_threshold)

total_flux = 0
rxn_flux_tups = []
for rxn in h_abs_rxns + disp_rxns:
    flux = graph_edges_dict_simple[step_pair][rxn][timepoint_index]
    
    rxn_flux_tups.append((rxn, flux))

rxn_flux_tups = sorted(rxn_flux_tups, key=lambda tup: tup[1], reverse=False)
for tup in rxn_flux_tups:
    rxn = tup[0]
    flux = tup[1]
    if flux > flux_threshold:
        total_flux += flux
        print ''
        print "**********************************************************************************"
        print "rxn#{0}: {1}: flux = {2} ".format(rxn.index, str(rxn), flux) # 
        display(rxn) 
print "***************************************"
print ''
print "TOTAL flux from h_abs and disp is {0}.".format(total_flux)



The step you see currently is between the two species ('C2H4(8)', 'C2H5(3)')

Positive flux in below means positive production of 1st node of the pair.

This step have 4 H_Abstration and Disproportionation in total.

TO avoid too much printout, the reactions shown below have fluxes above 1e-09.

**********************************************************************************
rxn#17: [CH]=C(13) + C[CH2](3) <=> C=C(8) + C=C(8): flux = 1.920494e-06 



**********************************************************************************
rxn#9: C[CH2](3) + [H](4) <=> [H][H](12) + C=C(8): flux = 6.27199e-06 



**********************************************************************************
rxn#6: [CH3](2) + C[CH2](3) <=> C=C(8) + C(6): flux = 8.960319e-06 



**********************************************************************************
rxn#7: C[CH2](3) + C[CH2](3) <=> C=C(8) + ethane(1): flux = 1.79485e-05 


***************************************

TOTAL flux from h_abs and disp is 3.5101303e-05.


## [user input] Functionality 3: find paths towards a particular species

Similar to functionality 2, please specify

Please specify

- target species

- depth of pathway you want search, e.g., depth = 2 means search pathways 2 steps up beyond target species

- path_top_list, e.g., [0, 3] means 1st step of the pathway takes the most significant branch and 2nd step would be the third most siginificant branch

In [25]:
target = "C#C(25)"
depth = 2
path_top_list = [0, 0]

In [21]:
current_node = target
print ''
print '\t Pathway Report '
print '======================================'
print 'The pathway you are intested in has {0} steps.'.format(depth)
print ''
print "total product flux for {0} is {1}.".format(target, spc_total_rop_dict[target][1][timepoint_index])
for step in range(depth):
    print "\n"
    prev_nodes = []
    for node1 in G:
        if current_node in G[node1]:
            prev_nodes.append(node1)
    prevNode_flux_list = [(prev_node, G[prev_node][current_node]['total_flux']) for prev_node in prev_nodes]
    sorted_prevNode_flux_list = sorted(prevNode_flux_list, key=lambda tup: -tup[1])
    
    # choose the top one as next node
    tup = sorted_prevNode_flux_list[path_top_list[step]]
    prev_node = tup[0]
    step_flux = tup[1]
    
    
    
    print ""
    print "Step{0} \t\t\t\t Flux".format(step+1)
    print '******************************************'
    
    print "{0} <-- {1}: \t\t {2}".\
    format(current_node, prev_node, step_flux)
    print '******************************************'
    
    step_pair = (prev_node, current_node)
    if step_pair not in graph_edges_dict_simple:
        step_pair = (current_node, prev_node)
    
    h_abs_rxns = []
    disp_rxns = []
    Pdep_rxns = []
    for rxn in graph_edges_dict_simple[step_pair]:
        if isinstance(rxn, PDepReaction):
            Pdep_rxns.append(rxn)
        elif rxn.family == "H_Abstraction":
            h_abs_rxns.append(rxn)
        elif rxn.family == "Disproportionation":
            disp_rxns.append(rxn)
        else:
            print ''
            print "Example reaction: rxn#{0}".format(rxn.index)
            print ''
            print "{0}: flux = {1} ".format(str(rxn), graph_edges_dict_simple[step_pair][rxn][timepoint_index])
            display(rxn)
            
    if len(h_abs_rxns) > 0:
        print ''
        print "Example H_Abstraction: rxn#{0}(1/{1} H_Abs)".format(h_abs_rxns[0].index, len(h_abs_rxns))
        print ''
        print str(h_abs_rxns[0])
        display(h_abs_rxns[0])
    if len(disp_rxns) > 0: 
        print ''
        print "Example Disproportionation: rxn#{0}(1/{1} Disp)".format(disp_rxns[0].index, len(disp_rxns))
        print ''
        print str(disp_rxns[0])
        display(disp_rxns[0])
    if len(Pdep_rxns) > 0: 
        print ''
        print "Example Pressure-dependent Rxn: rxn#{0} (1/{1} Pdep_rxns): ".format(Pdep_rxns[0].index, len(Pdep_rxns)) 
        print ''
        print str(Pdep_rxns[0])
        display(Pdep_rxns[0])
    
    current_node = prev_node


	 Pathway Report 
The pathway you are intested in has 2 steps.

total product flux for C#C(25) is 0.0001685005.



Step1 				 Flux
******************************************
C#C(25) <-- C2H3(13): 		 0.0001685005362
******************************************

Example reaction: rxn#18

C#C(25) + [H](4) <=> [CH]=C(13): flux = 0.0001646951 



Example Disproportionation: rxn#22(1/4 Disp)

[CH]=C(13) + [CH]=C(13) <=> C=C(8) + C#C(25)





Step2 				 Flux
******************************************
C2H3(13) <-- C2H4(8): 		 0.0001682719323
******************************************

Example reaction: rxn#13

[CH]=C(13) + [H](4) <=> C=C(8): flux = -3.402726e-06 



Example H_Abstraction: rxn#14(1/3 H_Abs)

[CH]=C(13) + C(6) <=> C=C(8) + [CH3](2)



Example Disproportionation: rxn#17(1/2 Disp)

[CH]=C(13) + C[CH2](3) <=> C=C(8) + C=C(8)


### More detail for the H_Abstration and Disproportionation in last step of the pathway

In [22]:
print ''
print 'The step you see currently is between the two species {0}'.format(step_pair)
print ''
print 'Positive flux in below means positive production of 1st node of the pair.'
print ''
print 'This step have {0} H_Abstration and Disproportionation in total.'.format(len(h_abs_rxns + disp_rxns))

flux_threshold = 1e-9

print ''
print 'TO avoid too much printout, the reactions shown below have fluxes above {0}.'.format(flux_threshold)

total_flux = 0
rxn_flux_tups = []
for rxn in h_abs_rxns + disp_rxns:
    flux = graph_edges_dict_simple[step_pair][rxn][timepoint_index]
    
    rxn_flux_tups.append((rxn, flux))

rxn_flux_tups = sorted(rxn_flux_tups, key=lambda tup: tup[1], reverse=False)
for tup in rxn_flux_tups:
    rxn = tup[0]
    flux = tup[1]
    if flux > flux_threshold:
        total_flux += flux
        print ''
        print "**********************************************************************************"
        print "rxn#{0}: {1}: flux = {2} ".format(rxn.index, str(rxn), flux) # 
        display(rxn) 
print "***************************************"
print ''
print "TOTAL flux from h_abs and disp is {0}.".format(total_flux)


The step you see currently is between the two species ('C2H3(13)', 'C2H4(8)')

Positive flux in below means positive production of 1st node of the pair.

This step have 5 H_Abstration and Disproportionation in total.

TO avoid too much printout, the reactions shown below have fluxes above 1e-09.

**********************************************************************************
rxn#14: [CH]=C(13) + C(6) <=> C=C(8) + [CH3](2): flux = 4.93783e-05 



**********************************************************************************
rxn#16: [CH]=C(13) + [H][H](12) <=> C=C(8) + [H](4): flux = 0.0001448447 


***************************************

TOTAL flux from h_abs and disp is 0.000194223.
